In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50

In [2]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
x_train.shape[1:]

(32, 32, 3)

In [4]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [5]:
from tensorflow.python.framework import tensor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = x_train.shape[1:])
# 가중치 동결하지 않는 경우 기존 가중치와 함께 학습이 되어서
# 훈련데이터가 충분하지 않으면 기존 가중치의 영향에 묻혀서 제대로 반영이 안될수 있음

# 기존 모델의 가중치를 동결 해제
for layer in base_model.layers:
  layer.trainable = True

# 마지막에 우리 분류기 넣는다.
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss = tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
1563/1563 [==============================] - 134s 52ms/step - loss: 1.3729 - accuracy: 0.5366 - val_loss: 2.1342 - val_accuracy: 0.2926
Epoch 2/10
1563/1563 [==============================] - 79s 50ms/step - loss: 1.1867 - accuracy: 0.5924 - val_loss: 1.4597 - val_accuracy: 0.5356
Epoch 3/10
1563/1563 [==============================] - 76s 49ms/step - loss: 1.1072 - accuracy: 0.6179 - val_loss: 2.2523 - val_accuracy: 0.3471
Epoch 4/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.1091 - accuracy: 0.6215 - val_loss: 1.2881 - val_accuracy: 0.5810
Epoch 5/10
1563/1563 [==============================] - 76s 48ms/step - loss: 0.9229 - accuracy: 0.6838 - val_loss: 0.8686 - val_accuracy: 0.7041
Epoch 6/10
1563/1563 [==============================] - 76s 49ms/step - loss: 0.8893 - accuracy: 0.6970 - val_loss: 1.5089 - val_accuracy: 0.4755
Epoch 7/10
1563/1563 [==============================] - 75

In [6]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.5142 - accuracy: 0.8279 - val_loss: 1.3982 - val_accuracy: 0.5598
Epoch 2/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.4812 - accuracy: 0.8388 - val_loss: 0.8973 - val_accuracy: 0.7161
Epoch 3/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.5603 - accuracy: 0.8113 - val_loss: 0.8235 - val_accuracy: 0.7387
Epoch 4/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.6036 - accuracy: 0.8040 - val_loss: 0.8789 - val_accuracy: 0.7136
Epoch 5/10
1563/1563 [==============================] - 79s 50ms/step - loss: 0.4750 - accuracy: 0.8404 - val_loss: 0.9668 - val_accuracy: 0.6954
Epoch 6/10
1563/1563 [==============================] - 78s 50ms/step - loss: 0.6052 - accuracy: 0.7958 - val_loss: 1.1107 - val_accuracy: 0.6619
Epoch 7/10
1563/1563 [==============================] - 76s 49ms/step - loss: 0.5756 - accuracy: 0.8035 - val_loss: 1.0472 -

In [7]:
model.save('resnet50_cifar10.h5')
model = keras.models.load_model('resnet50_cifar10.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import numpy as np
from PIL import Image
from glob import glob

In [9]:
img_files = glob('./test_imgs/*.jpg')
img_files[:3]

['./test_imgs/12.jpg', './test_imgs/07.jpg', './test_imgs/01.jpg']

In [10]:
img = Image.open(img_files[0])
x_tests = []
x = np.array(img.resize([32,32])) / 255.0
x_tests.append(x)
x_tests = np.array(x_tests)
np.argmax(model.predict(x_tests))

1/1 [==============================] - 2s 2s/step


7

In [11]:
classs_names = ['airplane',
'automobile'							,
'bird'										,
'cat'										,
'deer'									,
'dog'										,
'frog'									,
'horse'									,
'ship'									,
'truck']
classs_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [12]:
import os
tests = []
origin_images = []
for file in img_files:
  img = Image.open(file)
  origin_images.append(img)
  x = np.array(img.resize([32,32])) / 255.0
  tests.append(x)
tests = np.array(tests)
print(tests.shape)
preds = model.predict(tests)
print(len(preds))
pred_index =  [np.argmax(pred) for pred in preds]
print(pred_index)
for i, idx in enumerate(pred_index):  # i는 예측한 이미지의 순서 idx는 해당 클래스
  if not os.path.isdir(classs_names[idx]):
    os.mkdir(classs_names[idx])

  os.chdir(classs_names[idx])

  origin_images[i].save( img_files[i].split('/')[-1] )
  os.chdir('../')

(12, 32, 32, 3)
1/1 [==============================] - 1s 589ms/step
12
[7, 6, 0, 6, 8, 8, 4, 8, 2, 4, 1, 3]
